# Implementation of Auto-Encoding Variational Bayes

Vidvat Ramachandran & YueMing Shen

April 30, 2020

## Abstract

We implement the Variational Autoencoder (VAE) algorithm described in the paper *Auto-Encoding Variational Bayes* by Kingma and Welling (2013). This is a stochastic inference and learning algorithm which aims to improve learning efficiency under directed probablistic models with continuous latent variables and intractable posterior distributions. It is also scalable to large data sets which is commonly encountered in modern machine learning. 

In this report, we will first introduce the algorithm, its applications, advantages and limitations. Then in section two we will go through our model implementation which includes testing, profiling and optimization. In section three we demonstrate applications of our work on two real-world datasets, followed by comparisons with other competing algorithms. We then conclude with comments on the algorithm as well as thoughts for future developments.

## 1. Introduction


One major division in machine learning is generative versus discriminative modeling. While discriminative modeling can offer efficient algorithms with high precision, and provide users with flexible mapping from inputs to outputs, generative modeling is always attractive as it enables users to infuse expert knowledge in the model. The resulting model is usually highly intuitive and interpretable and it naturally allows for causal relations which makes the model generalize more effectively to new situations. Such modeling is generally known as unsupervised representation learning, and VAE is one of the algorithms which have been extensively employed for this purpose. 

VAE mainly tackles two issues in unsupervised learning of complicated distributions, large data sets, and intractable posterior distribution in directed probabilistic models with continuous latent variables. By introducing a reparameterization of the variational lower bound, the loss function under VAE can be efficiently optimized using standard stochastic gradient descent methods (SGD); and by fitting approximating inference model, especially the proposed use of neural networks for posterior inference, it marries graphical models with deep learning and improves learning efficiency.

### 1.1 Problem Scenario

Consider a large dataset $X = \{x^{(i)}\}_{i=1}^N$ with $N$ i.i.d. samples, and assume the data is generated by some random process involving continuous latent varialbe $z$ which follows the prior distribution $p_{\theta}(z)$. To build a good classifer/ discriminative model requires learning the posterior distribution of $p_{\theta}(z | x)$. And to further make the model generative requires knowledge on the distribution of $p_{\theta}(x | z)$. Because $z$ is unobserved, the parameters and distributions are largely unknown to us.

Modern machine learning tasks are usually high-dimensional, complicated and involves large datasets. Two examples given by the paper are recognition of human faces and hand-written digits. In these cases the inputs are high-dimensional images data with distribution of latent variables completely unknown to us. Two common problems faced by practitioners are:

1. Intractability: both the integral of the marginal likelihood $p_\theta(x) = \int p_\theta(z) p_\theta(x\mid z)dz$ and the true posterior density $p_\theta(z \mid x) = p_\theta(x \mid z) p_\theta(z)/p_\theta(x)$ are typically intractable for complicated distributions, which makes traditional machine learning methods like EM (expectation maximization) infeasible;
    
2. large data set: large amount of high dimensional data makes batch optimization or traditional Bayesian sampling approach like Markov-Chain Monte Carlo too expensive to implement.

VAE is introduced as a potential solution to these problems.

### 1.2 Algorithm Description


VAE consists of two independentaly parameterized models:

1. the recognition model to read in the data and use Bayes rule to encode it as knowledge of the distribution of latent variable (hence the name encoder). This is a discriminator that reverses the generative process. In order to address the intractability issue, the recognition model $q_{\phi}(z|x)$ is introduced as an approximation to the true posterior $p_{\theta}(z|x)$;
2. the generative model $p_\theta(x | z)$ which takes sampled latent variables and decodes them into meaningful representation of the data (hence the name decoder).

#### 1.2.1 Variational Lower Bound
We can start by looking at the log marginal Likelihood of data set $X$:

\begin{align*}
    \log(\prod_{i=1}^N p_{\theta}(x^{(i)})) = \sum_{i=1}^N \log p_{\theta}(x^{(i)}),
\end{align*}

where the log likelihood for each data point $i$ can be further written as:

\begin{align*}
    \log p_{\theta}(x^{(i)}) &= D_{KL}[q_{\phi}(z|x^{(i)}) || p_{\theta}(z|x^{(i)})] + \mathcal{L}(\theta, \phi; x^{(i)}).  
\end{align*}

The first term here is the KL divergence which measures the distance between the approximate distribution $q_{\phi}(z|x^{(i)})$ from the true posterior. Because KL divergence is non-negative, the second term is thus called the *variational lower bound* of the marginal likelihood for datapoint $i$. By maximizing the variational lower bound, we are able to bring our approximate distribution $q_{\phi}(z|x^{(i)})$ closer to the true posterior and maximize the model likelihood.

The variational lower bound can be written in two equivalent forms:
\begin{align*}
    \mathcal{L}(\theta, \phi; x^{(i)}) &= \mathbb{E}_{q_{\phi}(z|x^{(i)})}[\log p_{\theta}(x^{(i)},z) - \log q_{\phi}(z|x^{(i)})] \\
    &= -D_{KL}[q_{\phi}(z|x^{(i)}) || p_{\theta}(z)] + \mathbb{E}_{q_{\phi}(z|x^{(i)})}[\log p_{\theta}(x^{(i)}|z)].
\end{align*}

Ideally we would want to calculate the derivatives of the variational lower bound w.r.t. both the variational parameters $\phi$ and the generative parameters $\theta$ and then implement gradient descent. However, because the latent variable $z$ itself is sampled from the posterior distribution, this approach is infeasible and exhibits very high variance.

#### 1.2.2 Reparameterization Trick

By reparameterizing the latent variable, we are able to turn it into a deterministic variable $z = g_\phi(\epsilon, x)$ which relies on an auxiliary variable $\epsilon$ with independent marginal distribution $p(\epsilon)$. For example, in the case where we assume $z \sim N(\mu, \sigma^2)$, we can reparameterize it as $z = \mu + \sigma \epsilon$ where $\epsilon \sim N(0,1)$. The sampling can then be done on $\epsilon$. With this, the Monte Carlo estimate of the expectation is differentiable w.r.t. $\phi$ and hence SGD can be implemented. Proof and suggestions on reparameterizations under different scenarios can be found in Section 2.4 of the paper.

#### 1.2.3 SGVB

Applying the reparameterization trick to both formulation of the variational lower bound, we get two equivalent Stochastic Gradient Variational Bayes (SGVB) estimators.

\begin{align*}
    \hat{\mathcal{L}}^A(\theta, \phi;x^{(i)}) &= \frac 1L \sum_{l=1}^L \log p_{\theta}(x^{(i)}, z^{(i,l)} - \log q_{\phi}(z^{(i,l)} | x^{(i)})), \quad \mbox{where } z^{(i,l)} = g_{\phi}(\epsilon^{(i,l)}, x^{(i)}) \; \mbox{and } \epsilon^{(l)} \sim p(\epsilon) \\
    \hat{\mathcal{L}}^B(\theta, \phi;x^{(i)}) &= -D_{KL}[q_{\phi}(z|x^{(i)}) || p_{\theta}(z)] + \frac 1L \sum_{l=1}^L \log p_{\theta}(x^{(i)}|z^{(i,l)})
\end{align*}

Estimator A applies to all general cases. Estimator B is useful for cases where the KL divergence term has closed form solution which is usually the case since the user can choose the approximate model $q_{\phi}(z|x)$ and it is well behaved.

We can then implement the following algorithm for SGD as specified in section 2.3 of the paper.

**Algorithm**
1. $\theta$, $\phi$: Initialize parameters 
2. repeat until convergence of ($\theta$, $\phi$)
    - $X^M$: Random mini-batch of $M$ data points (drawn from the full dataset)
    - $\epsilon$: Random samples from noise distribution $p(\epsilon)$
    - grad: Gradiant calculation based on minimatch SGVB estimator (either estimator A or B can be used)
    - $\theta$, $\phi$: Gradient descent parameter update
3. return $\theta$, $\phi$

#### 1.2.4 AEVB

From the equation for estimator B above, i.e., objective function for SGD, we can see the connection to the auto-encoders (AEVB). Taking the negative of the second term would give us the expected reconstruction error which makes the model learn to decode back to a good representation of the original data; while the first term acts as regularization term which prevents the model deviate too much from the prior distribution of our belief.

### 1.2 Applications

VAE has emerged as one of the most popular algorithms in modern machine learning. It has been applied to different areas in generating various kinds of complicated data. In addition to the examples given in the paper for recognizing and generating hand-written digits and human faces, we have also seen applications of VAE in recognizing house numbers, the CIFAR images dataset which consists 80 million tiny images of 10 different classes as well as 3D scenery images (refer to reference #2).

There are various interesting possible applications of VAE. One such example is given in reference #7 that VAE can even be used to generate floor plans for a building of arbitrary shape. 

### 1.3 Advantages and Limitations

Advantages of VAEs are obvious

1. To start with, it is built on top of deep learning tools like neural networks and thus able to learn complicated non-linear relationships;
2. With the reparameterization trick, it can be trained using SGD and conveniently applied to large data sets;
3. It is a generative model which means that it is able to capture unobserved dependencies between data points and generalize to new problems;
4. There is a clear and recognized way to evaluate the quality of two models, i.e., through evaluation on the objective function, as opposed to some other generative models like GAN that it isn't clear how to compare two models. 

One limitation of VAEs is, because of the noises introduced by the reparameterization trick and the set up of standard decoders, the generated samples from VAEs are usually more blurred compared to those from some other model like GANs.

## 2. Implementation

### 2.1 Baseline codes

As a start, we did an end-to-end implementation of AVEB and SGVB using plain Python (hereinafter "Model v0"). Following the example given in paper Section 3 and Appendix C, our model uses a multivariate Gaussian Multilayer Perceptron (MLP) encoder, and Bernoulli MLP decoder, both with one single hidden layer. We assume the prior distribution for the latent variable $z$ is the centered isotropic multivariate Gaussian, and the variational approximate posterior a multivariate Gaussian with diagonal covariance structure. This is suitable for binary data or data which resembles a binary structure. For example, images which are mostly black and white can be represented as mostly either $0$ or $1$ after normalized by the constant $255$. 

We rely on `numpy` for numerical calculations. Three layers' looping were used. For each mini-batch, we loop through all the sample data in the batch, and for each data point we loop through all the latent variables sampled from the posterior distribution in order to calculate the total loss and gradients. We followed the recommendations in the paper to mini-batch of size $M=100$ and number of samples per data point $L=1$.

We used Xavier initialization (refer to reference #6) for weights and all zeros for bias.

In Model v0, vanilla SGD was used for optimization.

*Refer to Section 4 in the Model Development Notebook (link enclosed in the Appendix; hereinafter "MDN") for relevant codes and workings."*

### 2.2 Testing and Checking

In order to make sure our baseline codes are correct before optimization, we did a number of tests and checkings. 

*Refer to Section 8 in MDN for relevant codes and workings.*

#### 2.2.1 Gradient Checking

Our model involves heavy codings for gradient calculations with backpropagations on total $10$ martix form parameter variables (i.e., weights and bias parameters in Appendix C of the paper). Gradient descent is the key to this model implementation. In order to make sure the gradients are calculated correctly, we did gradient checking as suggested by Andrew Ng in his famous Machine Learning open course. 

To do gradient checking, we first use our function to calculate gradients for each matrix variable based on any set of data and model parameters. Then for each variable we do two more runs, one by adding a small number $\delta = 0.00001$ and one deducting $\delta$ from each element of this variable, while holding all the other variables unchanged. We can then "manually" calculate the gradients for each variable from changes in total loss from the two runs.

Our checking results show consistency of up to at least 5 decimal places for all the variables.

#### 2.2.2 Performance Reasonableness Checking

In addition to gradient checking, we also used Model v0 on real dataset for performance reasonableness checking.

Following the paper, we decided to use the MNIST dataset. It is a well-known dataset with large number of hand-written digits for machine learning. Because the images are all black and white, the model we built with Binary MLP decoder is suitable for it. We trained Model v0 with $36,000$ iterations of mini-batches of size $100$, $L=1$, with dimension of latent variable set to $2$. Results are shown below in Figure 1 and 2.

<img src="Figs/Fig1.png" style="width:600px;height:400px"/>
<center>Figure 1: Model v0 total loss (36,000 iterations)</center>

<img src="Figs/Fig2.png"/>
<center>Figure 2: Model v0 comparison between sample (left) and reconstructed (right) data (36,000 iterations)</center>

As we can see from Figure 1, the total loss has been consistently decreasing over the $36,000$ iterations. And from Figure 2 we can see that although the model has not yet learned very well for all the numbers (e.g., to properly distinguish between 0 vs 3, 7 vs ) and the images are blurry, it is obvious that the model has been learning from the data. These are all evidences that Model v0 was correctly built.

#### 2.2.2 Regression Test

As we will be doing model optimization, in order to make sure that optimized functions are correct, we have built in functionalities for regression testing vs results in Model v0. For each of the later models we explored, we always do regression testing to ensure model accuracy before proceeding to any further analysis. This ensures that all later versions of the model are correct and effectively prevents unintended errors during optimization.

### 2.3 Algorithm Optimization

During the performance checking on Model v0, we noticed that it is quite inefficient. It took an overnight run to complete the $36,000$ iterations. And also because of the nature of vanilla gradient descent, we can only stick to relatively small learning rate. We have tried and realized that a learning rate of $\alpha=0.005$ will already make the system unstable and encounter numerical problems. As a result, it converges/ learns rather slowly. Therefore we need to proceed to model optimization. Before we consider any performance optimization, we first explored improvement in optimization algorithm. 

**ADAM** is short for "Adaptive Moment Estimation". It is developed by the same group of researchers who wrote the paper for VAE. It effectively combines the ideas of momentum, RMSprop and bias correction in gradient descent implementation and has been the most widely adopted gradient descent method in current deep learning practice. 

The VAE paper used ADAM to implement SGD, and from our research on other online implementations of VAE, we noticed that most (if not all) of them use ADAM. Therefore we implemented ADAM for SGD (hereinafter "Model v1").

<img src="Figs/Fig3.png" style="width:600px;height:400px"/>
<center>Figure 3: Model v1 total loss for 10,000 iterations</center>

Comparing Figure 3 vs Figure 1 we can see that ADAM allows for much faster convergence. Also we were able to use larger learning rates (tested $\alpha = 0.005$ and $0.01$) without triggering any numerical issues.

<img src="Figs/Fig4.png"/>
<center>Figure 4: Model v1 comparison between sample (left) and reconstructed (right) data (60,000 iterations)</center>

Figure 4 shows a comparison of sample and reconstructed data under $60,000$ iterations. Although the set up is different from Figure 2, we can see clearly that the images here are sharper and the model is already able to distinguish between most of the numbers.

*Refer to Section 5 in MDN for relevant codes and workings, Section 9 for graphs.*

### 2.4 Profiling

Before performance optimization, we first did profiling on Model v1 to understand the bottlenecks of our codes. Here are the profiling results for 5 most time-consuming function calls.

In [6]:
import pstats
profile_filename = "Files/2020_04_24_11_18_59_Model_v1.1_profiling_v1.prof"
p = pstats.Stats(profile_filename)
p.sort_stats('time', 'cumulative').print_stats(5)
pass

Fri Apr 24 15:20:34 2020    Files/2020_04_24_11_18_59_Model_v1.1_profiling_v1.prof

         466012 function calls (464012 primitive calls) in 93.961 seconds

   Ordered by: internal time, cumulative time
   List reduced from 74 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100   80.268    0.803   81.196    0.812 <ipython-input-11-993a77bdd1ab>:1(grad)
        1    4.939    4.939   93.957   93.957 <ipython-input-17-e38d5ace111a>:1(train_ADAM)
    10000    2.440    0.000    2.655    0.000 <ipython-input-8-0aeb9911c8e9>:1(encoder_forward)
    10000    2.076    0.000    2.581    0.000 <ipython-input-10-317b689125af>:1(decoder_forward)
      100    1.382    0.014    1.927    0.019 <ipython-input-7-fd1817a619e0>:1(total_loss)




As expected, the most time consuming function is gradient calculations. With this in mind, we proceed to focus on performance optimization of the *grad* function.

*Refer to Section 6 in MDN for general profiling codes.*

### 2.5 Performance Optimization

For performance optimization, we implemented vectorization and Just-in-time (JIT) compiler Numba. Because the bottleneck of our model is gradient descent which is a sequential updating process, we did not consider paralellization.

#### 2.5.1 Vectorization

Our first attempt was vectorization. As explained, Model v0 and v1 both have 3 layers of for loops. We started by removing the inner loop, i.e., loop for sampling latent variables. There was some improvement in performance but not obvious. We then realized that because most of the time we are following the recommendation from the paper and set $L=1$, i.e., this loop only has 1 iteration, as a result, vectorization in this dimension will not significantly change the model performance.

We then swapped the loop implementation (as implied in table *Algorithm 1* in the paper, the order of these two loops can be changed) and instead vectorized the looping through sample data in each mini-batch. This model (hereinafter "Model v2") gives us significant improvement in model performance (refer to Figure 5 and Figure 6 below).

<img src="Figs/Fig5.png"/>
<center>Figure 5: Traing timing comparison between Model v1 and Model v2 for 10 mini-batches</center>

<img src="Figs/Fig6.png"/>
<center>Figure 6: Grad function timing comparison between Model v1 and Model v2</center>

Although in our VAE implementation, most of the time we set parameter $L$ to $1$ during training, we have decided to keep the additional dimension in the model to allow for flexibility for users to sample more than one latent variables per data point.

And we chose not to proceed for further vectorization because:

1. vectorization along both dimension will result in calculations involving 3-D matrix. As we intend to leverage Numba for further performance optimization and Numba nopython does not work with 3-D matrix, we decide not to do that
2. since $L=1$ most of the time, further vectorization along this dimension will not give us much performance improvements

*Refer to Section 7.2 in MDN for relevant codes and workings.*

#### 2.5.2 Numba

After vectorization, we further explored JIT compilation using package Numba. We tried different versions of codes in order to maximize its performance under *nopython* mode. Specifically,

1. We make sure our codes compile with Numba nopython as much as possible. For example, we use *reshape* for broadcasing so that Numba is able to interpret in nopython mode. However, we also realize that Numba nopython mode does not allow the passing of 3-D matrix which is required in our model implementation. As a result, two of the functions cannot be jitted under nopython mode;
1. We removed the use of heterogeneous list of passing variables in between all intermediate function calls so as to use nopython. We did not remove the use of these lists in the *train_AEVB" function with considerations on code readability and maintenance, also because this function contains passing of 3-D matrix which already violates nopython syntax;
2. All the functions are set to *cache=True* whenever we can so that JIT does not need to re-complile the codes.

<img src="Figs/Fig7.png"/>
<center>Figure 7: Traing timing comparison between Model v2 and Model v3 for 10 mini-batches</center>

<img src="Figs/Fig8.png"/>
<center>Figure 8: Grad function timing comparison between Model v2 and Model v3</center>

As we can see from Figure 7 and 8 above, JIT gives us some further performance improvement, and this is the prelim-final version of our codes (Hereinafter "Model v3"). The only difference between our final codes and Model v3 is the removal of helper functions used in development stage, addition in documentation and changes to make the function more user-friendly.

*Refer to Section 7.5 in MDN for relevant codes and workings. Note that during model development stage, we tried total 7 models. We did not include all profiling and timing results in this report. Refer to Section 7 in MDN for details.*

#### 2.5.3 Limitation and future developments

In terms of performance optimization, here we document the limitations as well as potential future developments.

1. Our model implementation heavily relies on `numpy` for matrix related calculations. Therefore if time allows, we are interested to try the package `numexpr`;
2. Another possible optimization direction is to try writing the basic matrix operations like multiplication and element-wise matrix operations using C++. Theoretically this should be able to further improve the model performance.
3. We were not able to compile Numba JIT into full nopython mode because the current model implementation involves 3-D matrix. This not only potentially slows down the model, but also leads to release of warning messages in the final package. We considered two possible ways to solve this issue:

    - force parameter $L=1$, i.e., 1 sample of latent variable in the posterior distribution. This will help us get rid of the 3D matrices;
    - use brute force to turn off all system warnings (based on our research, there is no other way to suppress the nopython warnings)

After weighing the pros and cons, we have decided to keep it as it is (i.e., with Numba nopython warnings in the package codes) to allow for model flexibility and robustness. However this can be further improved.

## 3. Application and Comparison

### 3.1 MNIST Dataset

We implemented our final model on the MNIST dataset. Figure 9, 10 and 11 below are the reconstructed vs original images based on parameters from three model runs, each with 100k iterations of mini-match of size 100 with dimension of latent space variables set to $3$, $5$ and $10$ respectively. We can see that given sufficient training time, the models are able to distinguish between all the numbers. And generally higher dimension of the latent variable space gives sharper images.

<img src="Figs/Fig9.png"/>
<center>Figure 9: Comparison between sample (left) and reconstructed (right) data (100,000 iterations, dz = 3)</center>

<img src="Figs/Fig10.png"/>
<center>Figure 10: Comparison between sample (left) and reconstructed (right) data (100,000 iterations, dz = 5)</center>

<img src="Figs/Fig11.png"/>
<center>Figure 11: Comparison between sample (left) and reconstructed (right) data (100,000 iterations, dz = 10)</center>

*Refer to Section 9 in MDN for relevant codes and workings for graphs.*

### 3.2 Caltech Dataset

We further tested our model on another real world data set which is not included in the paper. It is the CalTech 101 Silhouettes Data Set which is a collection different object silhouettes. Similar to the MNIST data, it's mostly black and white and thus suitable for the model we built with Bernoulli decoder. Due to time contraints, we did not spend a long time training this set of data. However, from Figure 12 below we can see that with 60,000 interations, our model has managed to learn and reconstruct the different types of object silhouettes.

<img src="Figs/Fig12.png"/>
<center>Figure 12: Comparison between sample (left) and reconstructed (right) data (60,000 iterations, dm = 128, dz = 3)</center>

*Refer to Section 9 in MDN for relevant codes and workings for graphs. Refer to Reference #10 for links and detailed descriptions to the dataset.*


## 4. Comparative Analysis

VAE is a generative model. It's main aim is to learn some features(latent) from the data and then use these features to generate new data that looks like it came from the actual data set. More specifically, it learns to generate data which can be said to be generated from its true distribution. As such, after training on the MNIST data set, the next step is to generate some 'fake' images that look like they were part of the handwritten data set. 

We look at a couple of other implementations of generative models, one is another implementation of the Variational Autoencoder, a Convolutional Variational Autoencoder(CVAE) and a different model, known as GANs - Generative Adversarial Networks, in this case we are using Deep Convolutional Generative Adversarial Network(DCGAN). These implementations were taken from TensorFlow Tutorials. The two implementations were run on Google Colab using the GPU.

Since, all these models take a lot of time to run and preferably should be run on systems with high computational power or GPUs, we will not discuss the time these take to run. Another reason is that more efficient implementation of the VAE would be done using other modules such as TensorFlow, which we do not use in our implementation, hence it is bound to slower.

A comparison of the optimization routine is also not possible here, while the VAE implementations have the ability to calculate the log-likelihoods and the lower-bounds, GANs in general do not. Hence, as in most cases, comparison between generative models is done by simple visual inspection. We show here, images generated by the three models, after certain amount of training.

<table><tr>
    <td> <img src="Figs/vae_gen.png" alt="Drawing" style="width: 190px;"/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img src="Figs/cvae_gen.png" alt="Drawing" style="width: 150px;"/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img/> </td>
    <td> <img src="Figs/dcgan_gen.png" alt="Drawing" style="width: 150px;"/> </td>
</tr></table>

<p></p>
<center>Figure 13: Images created by the generative models VAE(left), CVAE(center), DCGAN(right)</center>
<p></p>
<p></p>
Note that the codes were run for a relatively very short time period as compared to a full analysis. The CVAE implementation ran for a bit longer than the DCGAN implementation.
The main differences between the images we can observe is that the VAE variants seem to produce blurry or rather noisy images, while GAN produces a relatively sharper image. The CVAE implementation clearly seems a bit less noisy than our VAE implentation, but the VAE digits look more recognizable.

GANs and VAEs are quite popular as generative models and both have their pros and cons. GANs are more of a black-box, while the inherent sampling method behind VAE introduces this blurry like filter on the images, which are primarily a consequence of the cost functions chosen. Another popular generative model is the Restricted Boltzmann machine, though the image generation in that model is a bit complicated, and we could not find a nice implementation so it's ommited here.

## 5. Conclusion

In this project we studied and implemented the VAE algorithm. Through this process we've learned the power of the auto-encoder -- given sufficient time and data, how they are able to learn to recognize numbers and even faces. We see how VAE is able to combine deep learning discriminative models such as neural networks with generative modelings and make it a more efficient machine learning tool. We also recognized the well-recognized limitation of VAE. Because of the noises introduced from the reparameterization tricks, the images generated by VAEs tend to more blurry. We are interested to explore alternative generative models like GANs to further explore their strengths and weaknesses.

VAEs and in general all neural networks have to be implemented differenty depending on the data set. The current implementation is based on the MNIST data set which has a specific structure, changing the data would require changing the neural networks and the gradients as well. Since this is a simple implementation, we were not able to test other data sets. A better implementation would be to use modules specifically made for such purposes such as TensorFlow or PyTorch, they would allow for more flexibility and faster run times that just using numpy.

From working with high-dimensional dataset, we have come to appreciate the importance of algorithm and performance optimization. If time allows, we would be interested in further explorations in alternative approaches to improve the model efficiency.

## References

1. D.P.Kingma, M.Weling. *Auto-Encoding Variational Bayes*. ArXiv e-prints. 1312.6114.

2. C. Doersch. *Tutorial on Variational Autoencoders*. ArXiv e-prints. 1606.05908.

3. D.P.Kingma, J.Ba. *Adam: A Method for Stochastic Optimization*. ArXiv e-prints. 1412.6980.

4. D.P.Kingma. *An Introduction to Variational Autoencoders*. ArXiv e-prints. 1906.0269.

5. Coursera Machine Learning course by Andrew Ng. https://www.coursera.org/learn/machine-learning/home/welcome

6. Xavier initialization. https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79

7. Example applications of VAE. https://towardsdatascience.com/generating-images-with-autoencoders-77fd3a8dd368

8. Deep Convolutional Generative Adversarial Network - TensorFlow Tutorials. https://www.tensorflow.org/tutorials/generative/dcgan

9. Convolutional Variational Autoencode - TensorFlow Tutorials.. https://www.tensorflow.org/tutorials/generative/cvae

10. Caltech Dataset. https://people.cs.umass.edu/~marlin/data.shtml

## Code

Full implementation, references and workings are available at public GitHub repository https://github.com/RV29/VAE. Instructions on installation of the package is available in the README file. 

All the model development codings and supportings can be found in this notebook in the GitHub repository:
- Model Development/Model Development Notebook.ipynb

## Installation and use

To install the package use:

`pip install -i https://test.pypi.org/simple/ VAE-RV29`

The package has just the basic dependencies, `numpy`, `matplotlib` and `numba`

To use the modules, simply use:

`from VAE import VAE`

See the demo file, "VAE on MNIST - DEMO.ipynb" in the repository for a short demo